# Descripción

La compañía de seguros Sure Tomorrow quiere resolver varias tareas con la ayuda de machine learning y te pide que evalúes esa posibilidad.
- Tarea 1: encontrar clientes que sean similares a un cliente determinado. Esto ayudará a los agentes de la compañía con el marketing.
- Tarea 2: predecir la probabilidad de que un nuevo cliente reciba una prestación del seguro. ¿Puede un modelo de predictivo funcionar mejor que un modelo dummy?
- Tarea 3: predecir el número de prestaciones de seguro que un nuevo cliente pueda recibir utilizando un modelo de regresión lineal.
- Tarea 4: proteger los datos personales de los clientes sin afectar al modelo del ejercicio anterior. Es necesario desarrollar un algoritmo de transformación de datos que dificulte la recuperación de la información personal si los datos caen en manos equivocadas. Esto se denomina enmascaramiento u ofuscación de datos. Pero los datos deben protegerse de tal manera que no se vea afectada la calidad de los modelos de machine learning. No es necesario elegir el mejor modelo, basta con demostrar que el algoritmo funciona correctamente.


# Preprocesamiento y exploración de datos

## Inicialización

In [133]:
# pip install scikit-learn --upgrade

In [4]:
import numpy as np
import pandas as pd
import math

import seaborn as sns
# import sklearn.linear_model
from sklearn.linear_model import LinearRegression
import sklearn.preprocessing


from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.metrics import confusion_matrix, f1_score, mean_squared_error, r2_score
from sklearn.dummy import DummyClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from IPython.display import display

## Carga de datos

Carga los datos y haz una revisión básica para comprobar que no hay problemas obvios.

In [135]:
# df = pd.read_csv('/datasets/insurance_us.csv')

In [136]:
df = pd.read_csv('./insurance_us.csv')

Renombramos las columnas para que el código se vea más coherente con su estilo.

In [137]:
df = df.rename(columns={'Gender': 'gender', 'Age': 'age', 'Salary': 'income', 'Family members': 'family_members', 'Insurance benefits': 'insurance_benefits'})

In [138]:
df.sample(10)

,gender,age,income,family_members,insurance_benefits
200,1,36.0,23100.0,1,0
2059,0,27.0,49200.0,2,0
4608,1,41.0,32600.0,0,0
2481,0,38.0,50000.0,0,0
4662,1,47.0,32100.0,1,1
4171,1,47.0,43900.0,2,1
3826,1,42.0,42600.0,1,1
1721,1,29.0,27300.0,1,0
4072,1,27.0,44300.0,1,0
2867,1,32.0,48900.0,0,0


In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   gender              5000 non-null   int64  
 1   age                 5000 non-null   float64
 2   income              5000 non-null   float64
 3   family_members      5000 non-null   int64  
 4   insurance_benefits  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Cambiando el tipo de columna float a int.

In [140]:
df['age'] = df['age'].astype('int')

In [141]:
# comprueba que la conversión se haya realizado con éxito
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   gender              5000 non-null   int64  
 1   age                 5000 non-null   int32  
 2   income              5000 non-null   float64
 3   family_members      5000 non-null   int64  
 4   insurance_benefits  5000 non-null   int64  
dtypes: float64(1), int32(1), int64(3)
memory usage: 175.9 KB


Analizando estadisticas descriptivas

In [142]:
df.describe()

,gender,age,income,family_members,insurance_benefits
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


- Gender: Acording to the dataframe description, the mean and the standard deviation are similar. For this boolean columns basically it shows the distribution it - is half and half
- Age: The mean age is 31 years old, the standard deviation bewteen the ages is around 8 years old, and 75% of this dataset represent 37 yrs old or less.
- Income: The median income is $39,916.36, with a standard deviation of $9900. The third quartile shows an income of 46,600 or less. 
- Family Members: The median of the family members is just 1, having a maximum of 6 and minimum of 0, The 75% of the people in this dataset has 2 family members.
- Insurance Benefits: The median of the insurance benefits is 0.14 from wich the third quartile represents 0.

## Análisis exploratorio de datos

Vamos a comprobar rápidamente si existen determinados grupos de clientes observando el gráfico de pares.

In [143]:
g = sns.pairplot(df, kind='hist')
g.fig.set_size_inches(12, 12)

De acuerdo, es un poco complicado detectar grupos obvios (clústeres) ya que es difícil combinar diversas variables simultáneamente (para analizar distribuciones multivariadas). Ahí es donde LA y ML pueden ser bastante útiles. 

# Tarea 1. Clientes similares

En el lenguaje de ML, es necesario desarrollar un procedimiento que devuelva los k vecinos más cercanos (objetos) para un objeto dado basándose en la distancia entre los objetos.
Es posible que quieras revisar las siguientes lecciones (capítulo -> lección)- Distancia entre vectores -> Distancia euclidiana
- Distancia entre vectores -> Distancia Manhattan

Para resolver la tarea, podemos probar diferentes métricas de distancia.

Escribe una función que devuelva los k vecinos más cercanos para un $n^{th}$ objeto basándose en una métrica de distancia especificada. A la hora de realizar esta tarea no debe tenerse en cuenta el número de prestaciones de seguro recibidas.
Puedes utilizar una implementación ya existente del algoritmo kNN de scikit-learn (consulta [el enlace](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors)) o tu propia implementación.
Pruébalo para cuatro combinaciones de dos casos- Escalado
  - los datos no están escalados
  - los datos se escalan con el escalador [MaxAbsScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html)
- Métricas de distancia
  - Euclidiana
  - Manhattan

Responde a estas preguntas:- ¿El hecho de que los datos no estén escalados afecta al algoritmo kNN? Si es así, ¿cómo se manifiesta?- ¿Qué tan similares son los resultados al utilizar la métrica de distancia Manhattan (independientemente del escalado)?

In [144]:
def get_knn(data, n, k, metric):
    """
    Devuelve los k vecinos más cercanos

    :param data: DataFrame de pandas utilizado para encontrar objetos similares dentro del mismo lugar    
    :param n: número de objetos para los que se buscan los vecinos más cercanos    
    :param k: número de vecinos más cercanos a devolver
    :param métrica: nombre de la métrica de distancia    """
    
	#Defining the desired columns
	# Training the model
    model = NearestNeighbors(n_neighbors=k, metric=metric)
	# Fitting the model
    model.fit(data.to_numpy())
    
	# Executing kneighbors function to get an specific distance
    nbrs_distances, nbrs_indices = model.kneighbors([data.iloc[n]], k, return_distance=True)

    data_res = pd.concat([
        data.iloc[nbrs_indices[0]], 
        pd.DataFrame(nbrs_distances.T, index=nbrs_indices[0], columns=['distance'])
        ], axis=1)
    
    return data_res

Escalar datos.

In [204]:
feature_names = ['gender', 'age', 'income', 'family_members']

def escalate_df(df, cols):
    transformer_mas = sklearn.preprocessing.MaxAbsScaler().fit(df[cols].to_numpy())
    df_scaled = df.copy().astype(np.float64)
    df_scaled.loc[:, cols] = transformer_mas.transform(df[cols].to_numpy())
    return df_scaled

In [205]:
df_scaled = escalate_df(df,feature_names)
df_scaled.head(5)

,gender,age,income,family_members,insurance_benefits,insurance_benefits_received
0,1.0,0.630769,0.627848,0.166667,0.0,0.0
1,0.0,0.707692,0.481013,0.166667,1.0,1.0
2,0.0,0.446154,0.265823,0.000000,0.0,0.0
3,0.0,0.323077,0.527848,0.333333,0.0,0.0
4,1.0,0.430769,0.330380,0.000000,0.0,0.0


Ahora, vamos a obtener registros similares para uno determinado, para cada combinación

## Testing the code 

The code above will be tested with the following parameters.

In [147]:
# Parameters
k = 3
n = 5
metrics = ['euclidean', 'manhattan']

In [206]:
df[feature_names].iloc[n]

gender                1.0
age                  43.0
income            41000.0
family_members        2.0
Name: 5, dtype: float64

Ejecutando la funcion knn para los datos sin escalamiento, obteniendo las distancias (manhattan and euclidean) 

In [149]:
for metric in metrics:
	print(f'No Scaled Test, {metric} metric')
	print('___________________________',)
	print(get_knn(df[feature_names],k,n,metric))

No Scaled Test, euclidean metric
___________________________
      gender  age   income  family_members  distance
3          0   21  41700.0               2  0.000000
3894       0   21  41700.0               2  0.000000
2102       0   23  41700.0               0  2.828427
416        0   25  41700.0               2  4.000000
4872       1   26  41700.0               2  5.099020
No Scaled Test, manhattan metric
___________________________
      gender  age   income  family_members  distance
3          0   21  41700.0               2       0.0
3894       0   21  41700.0               2       0.0
416        0   25  41700.0               2       4.0
2102       0   23  41700.0               0       4.0
4872       1   26  41700.0               2       6.0


Ejecutando la funcion knn para los datos escalados, obteniendo las distancias (manhattan and euclidean) 

In [150]:
for metric in metrics:
	print(f'Scaled Test, {metric} metric')
	print('___________________________',)
	print(get_knn(df_scaled[feature_names],k,n,metric))

Scaled Test, euclidean metric
___________________________
      gender       age    income  family_members  distance
3894     0.0  0.323077  0.527848        0.333333  0.000000
3        0.0  0.323077  0.527848        0.333333  0.000000
29       0.0  0.323077  0.534177        0.333333  0.006329
1614     0.0  0.323077  0.518987        0.333333  0.008861
4125     0.0  0.307692  0.525316        0.333333  0.015592
Scaled Test, manhattan metric
___________________________
      gender       age    income  family_members  distance
3894     0.0  0.323077  0.527848        0.333333  0.000000
3        0.0  0.323077  0.527848        0.333333  0.000000
29       0.0  0.323077  0.534177        0.333333  0.006329
1614     0.0  0.323077  0.518987        0.333333  0.008861
4125     0.0  0.307692  0.525316        0.333333  0.017916


Respuestas a las preguntas

**¿El hecho de que los datos no estén escalados afecta al algoritmo kNN? Si es así, ¿cómo se manifiesta?** 

Si, al comparar los resultados obtenidos con o sin escala, es visible la diferencia en sus resultados, esto se refleja en la diferencia de puntos decimales entre los datos calibrados y no calibrados.

**¿Qué tan similares son los resultados al utilizar la métrica de distancia Manhattan (independientemente del escalado)?** 

Son totalmente diferentes, no solo por la separacion de tres puntos decimales sino además por el valor final.

# Tarea 2. ¿Es probable que el cliente reciba una prestación del seguro?

En términos de machine learning podemos considerarlo como una tarea de clasificación binaria.

Con el valor de `insurance_benefits` superior a cero como objetivo, evalúa si el enfoque de clasificación kNN puede funcionar mejor que el modelo dummy.
Instrucciones:

- Construye un clasificador basado en KNN y mide su calidad con la métrica F1 para k=1...10 tanto para los datos originales como para los escalados. Sería interesante observar cómo k puede influir en la métrica de evaluación y si el escalado de los datos provoca alguna diferencia. Puedes utilizar una implementación ya existente del algoritmo de clasificación kNN de scikit-learn (consulta [el enlace](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)) o tu propia implementación.- Construye un modelo dummy que, en este caso, es simplemente un modelo aleatorio. Debería devolver "1" con cierta probabilidad. Probemos el modelo con cuatro valores de probabilidad: 0, la probabilidad de pagar cualquier prestación del seguro, 0.5, 1.

La probabilidad de pagar cualquier prestación del seguro puede definirse como
$$
P\{\text{prestación de seguro recibida}\}=\frac{\text{número de clientes que han recibido alguna prestación de seguro}}{\text{número total de clientes}}.
$$

Divide todos los datos correspondientes a las etapas de entrenamiento/prueba respetando la proporción 70:30.

#### Calculando el objetivo

In [151]:
## La columna insurance_benefits muestra el número de prestaciones que ha usado el cliente, por lo que tenemos que transformarlo en un booleano para tener nuestro objetivo.
# Definiendo y
df['insurance_benefits_received'] = np.where(df['insurance_benefits']>0, 1,0)

#### Comprueba el desequilibrio de clases 

In [152]:
# comprueba el desequilibrio de clases con value_counts()
df['insurance_benefits_received'].value_counts()

insurance_benefits_received
0    4436
1     564
Name: count, dtype: int64

Tenemos mas datos en 0 que en 1 por lo que el algoritmo le otorgará mas peso al 0

In [153]:
def eval_classifier(y_true, y_pred):
    '''Esta función devolverá el score entre el valor predicho y el valor real'''
    f1 = f1_score(y_true,y_pred)
    # report = sklearn.metrics.classification_report(y_test,y_predict)
    # conf_matrix = sklearn.metrics.confusion_matrix(y_test,y_predict)
    print(f'F1 Score: {f1}')
    print('Matriz de confusión:')
    print(confusion_matrix(y_true, y_pred, normalize='all'))
    print('_____________________', end='\n\n')

In [243]:
def knn_score(X_train, X_test, y_train, y_test,k):
    '''Esta función entrenará un modelo de clasificación Kneighbors para predecir y obtener el f1 score '''
    model= KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    # print(y_predict) Porque estoy obteniendo el mismo predict????
    return eval_classifier(y_test, y_predict)

In [241]:
# Declarar una funcion ya que se necestia para los df normales como los escalados
def main(X,y,cols, escalated=False ):
	'''Esta funcion dividira los valores de entrada y balanceará el modelo de entrenamiento
	Finalmente pasara los valores a un modelo de vecinos para obtener la puntuación por el número de vecinos de 1 a 11'''
	# Dividimos nuestro conjunto en entrenamiento y test
	X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=1234)

	if escalated == True:
		X_train = escalate_df(X_train, cols)
		X_test = escalate_df(X_test, cols)

	# # Entrenamos el clasificador KNN
	for k in range(1,11):
		print(f'RESULTADOS PARA EL REPORTE CON k={k} vecinos: \n')
		knn_score(X_train, X_test, y_train, y_test,k)
		
	

F1 Score para un conjunto escalado

In [244]:
y = df['insurance_benefits_received']
X = df.drop(columns='insurance_benefits_received')
main(X,y, feature_names,escalated=True )


RESULTADOS PARA EL REPORTE CON k=1 vecinos: 

F1 Score: 1.0
Matriz de confusión:
[[0.88466667 0.        ]
 [0.         0.11533333]]
_____________________

RESULTADOS PARA EL REPORTE CON k=2 vecinos: 

F1 Score: 1.0
Matriz de confusión:
[[0.88466667 0.        ]
 [0.         0.11533333]]
_____________________

RESULTADOS PARA EL REPORTE CON k=3 vecinos: 

F1 Score: 1.0
Matriz de confusión:
[[0.88466667 0.        ]
 [0.         0.11533333]]
_____________________

RESULTADOS PARA EL REPORTE CON k=4 vecinos: 

F1 Score: 1.0
Matriz de confusión:
[[0.88466667 0.        ]
 [0.         0.11533333]]
_____________________

RESULTADOS PARA EL REPORTE CON k=5 vecinos: 

F1 Score: 1.0
Matriz de confusión:
[[0.88466667 0.        ]
 [0.         0.11533333]]
_____________________

RESULTADOS PARA EL REPORTE CON k=6 vecinos: 

F1 Score: 1.0
Matriz de confusión:
[[0.88466667 0.        ]
 [0.         0.11533333]]
_____________________

RESULTADOS PARA EL REPORTE CON k=7 vecinos: 

F1 Score: 1.0
Matriz de 

Podemos oservar en el conjunto escalado que ninguno de los valores fueron clasificados con Falsos positivos y falsos Negativos

F1 Score para un conjunto sin escalar

In [245]:
main(X,y, feature_names,escalated=False )

RESULTADOS PARA EL REPORTE CON k=1 vecinos: 

F1 Score: 0.6428571428571429
Matriz de confusión:
[[0.86066667 0.024     ]
 [0.04933333 0.066     ]]
_____________________

RESULTADOS PARA EL REPORTE CON k=2 vecinos: 

F1 Score: 0.37719298245614036
Matriz de confusión:
[[0.87666667 0.008     ]
 [0.08666667 0.02866667]]
_____________________

RESULTADOS PARA EL REPORTE CON k=3 vecinos: 

F1 Score: 0.4117647058823529
Matriz de confusión:
[[0.874      0.01066667]
 [0.08266667 0.03266667]]
_____________________

RESULTADOS PARA EL REPORTE CON k=4 vecinos: 

F1 Score: 0.21212121212121213
Matriz de confusión:
[[0.882      0.00266667]
 [0.10133333 0.014     ]]
_____________________

RESULTADOS PARA EL REPORTE CON k=5 vecinos: 

F1 Score: 0.208955223880597
Matriz de confusión:
[[0.88       0.00466667]
 [0.10133333 0.014     ]]
_____________________

RESULTADOS PARA EL REPORTE CON k=6 vecinos: 

F1 Score: 0.10810810810810811
Matriz de confusión:
[[0.88333333 0.00133333]
 [0.10866667 0.00666667]]
_

Para los resultados que no fueron estalados, obtenemos una incorrecta signación en falsos positivos y falsos negativos, afectando el F1 Score

#### Generando un modelo aleatorio

In [246]:
# generar la salida de un modelo aleatorio

def rnd_model_predict(P, size, seed=42):
    '''El objetivo de esta funcion simular los valores de y_predict de un df'''
    rng = np.random.default_rng(seed=seed) # Genero una semilla para que mis futuros randoms siempre sean los mismos en los análisis.
    return rng.binomial(n=1, p=P, size=size)

In [247]:
for P in [0, df['insurance_benefits_received'].sum() / len(df), 0.5, 1]:
    print(f'La probabilidad: {P:.2f}')
    y_pred_rnd = rnd_model_predict(P, len(df))
    eval_classifier(df['insurance_benefits_received'], y_pred_rnd)    

La probabilidad: 0.00
F1 Score: 0.0
Matriz de confusión:
[[0.8872 0.    ]
 [0.1128 0.    ]]
_____________________

La probabilidad: 0.11
F1 Score: 0.12072072072072072
Matriz de confusión:
[[0.7914 0.0958]
 [0.0994 0.0134]]
_____________________

La probabilidad: 0.50
F1 Score: 0.19807883405101026
Matriz de confusión:
[[0.456  0.4312]
 [0.053  0.0598]]
_____________________

La probabilidad: 1.00
F1 Score: 0.20273184759166069
Matriz de confusión:
[[0.     0.8872]
 [0.     0.1128]]
_____________________



Creando valores artificiales para y_predict podemos observar los cambios que tendría la informacion.

- Para probabilidad 0 : El 89% de los datos se clasificaron correctamente pero un 12% daría falsos '1', al no existir valores 1 reales no podemos tener verdaderos negativos.
- Para la probabilidad de 0.11% : Tendríamos una mayor cantidad de '0' y el modelo clasificaria la mayoría correctamente. Los falos negativos para 0 y 1 serían mayores que nuestros verdaderos negativos.
- Para la probabilidas de 0.5%: Tendríamos un balance entre ambos valores, en este caso podemos observar que el porcentaje de falsos positivos y falsos negativos aumento considerablemente, indicando que el modelo (supuesto) debería ser calibrado
- Para la probabilidad de 1: El modelo tiene el peso en los valores 1 por lo que los verdaderos positivos no existirían, en este caso se puede observar una predicción muy pobre del modelo (modelo supuesto).

# Tarea 3. Regresión (con regresión lineal)

Con `insurance_benefits` como objetivo, evalúa cuál sería la RECM de un modelo de regresión lineal.

Construye tu propia implementación de regresión lineal. Para ello, recuerda cómo está formulada la solución de la tarea de regresión lineal en términos de LA. Comprueba la RECM tanto para los datos originales como para los escalados. ¿Puedes ver alguna diferencia en la RECM con respecto a estos dos casos?

Denotemos
- $X$: matriz de características; cada fila es un caso, cada columna es una característica, la primera columna está formada por unidades- $y$ — objetivo (un vector)- $\hat{y}$ - objetivo estimado (un vector)- $w$ — vector de pesos
La tarea de regresión lineal en el lenguaje de las matrices puede formularse así:
$$
y = Xw
$$

El objetivo de entrenamiento es entonces encontrar esa $w$ w que minimice la distancia L2 (ECM) entre $Xw$ y $y$:

$$
\min_w d_2(Xw, y) \quad \text{or} \quad \min_w \text{MSE}(Xw, y)
$$

Parece que hay una solución analítica para lo anteriormente expuesto:
$$
w = (X^T X)^{-1} X^T y
$$

La fórmula anterior puede servir para encontrar los pesos $w$ y estos últimos pueden utilizarse para calcular los valores predichos
$$
\hat{y} = X_{val}w
$$

Divide todos los datos correspondientes a las etapas de entrenamiento/prueba respetando la proporción 70:30. Utiliza la métrica RECM para evaluar el modelo.

#### Creando Modelo de regresion linear

In [160]:
class MyLinearRegression:
    '''Esta clase identifica los pesos de una matriz a traves de su funcion fit y predice 
    "Y" a traves de la multiplicacion de el peso y la matriz de diseño extendida
    '''
    
    def __init__(self):
        
        self.weights = None
    
    def fit(self, X, y):
        X2 = np.append(np.ones([len(X), 1]), X, axis=1) #Determinando la matriz con columna de sesgo (intercepto)
        self.weights = np.linalg.inv(X2.T @ X2) @ X2.T @ y # Obteniendo los pesos

    def predict(self, X):
        X2 = np.append(np.ones([len(X), 1]), X, axis=1) #Determinando la matriz con columna de sesgo (intercepto)
        y_pred = X2 @ self.weights 
        
        return y_pred

In [161]:
def eval_regressor(y_true, y_pred):
    "Esta funcion toma los parametros y_true y y_pred pra obtener rmse y r2_score "
    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    print(f'RMSE: {rmse:.2f}')
    
    r2score = math.sqrt(r2_score(y_true, y_pred))
    print(f'R2: {r2score:.2f}')    
    

In [162]:
def linear_regression_report(X,y):
    cols = ['age', 'gender', 'income', 'family_members', 'intercept']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12345)
    lr = MyLinearRegression()
    lr.fit(X_train, y_train)
    w = lr.weights
    print(pd.DataFrame(w.reshape(1,-1), columns=cols), end='\n\n')
    y_test_pred = lr.predict(X_test)
    eval_regressor(y_test, y_test_pred)

Regresion linear para datos sin escalar

In [163]:

X = df[feature_names].to_numpy()
y = df['insurance_benefits'].to_numpy()

linear_regression_report(X,y)

        age    gender   income  family_members  intercept
0 -0.943539  0.016427  0.03575   -2.607437e-07   -0.01169

RMSE: 0.34
R2: 0.66


Con los datos sin escalar podemos observar un RMSE de 0.34, el cual detalla que pdemos esperar una variacion de 0..34 unidades del valor real. R2 nos muestra un valor de 0.66, es decir que en un 66% nuestra variable objetivo puede ser explicada/predicha por las colummnas de sus caracteristicas X

Regresion linear para datos escalados

In [164]:
y = df['insurance_benefits_received'].values
X = df_scaled.drop(columns='insurance_benefits').values

linear_regression_report(X,y)

        age    gender    income  family_members  intercept
0 -0.650583 -0.000731  1.608161        0.010817  -0.037975

RMSE: 0.23
R2: 0.66


Al escalar los datos obtenemos un resultado mas exacto, teniendo una probabilidad de 0.23 unidades del valor real. (0 o 1). Por otra parte R2 menciona que en un 66% nuestra variable objetivo puede ser explicada/predicha por las colummnas de X

# Tarea 4. Ofuscar datos

Lo mejor es ofuscar los datos multiplicando las características numéricas (recuerda que se pueden ver como la matriz $X$) por una matriz invertible $P$. 

$$
X' = X \times P
$$

Trata de hacerlo y comprueba cómo quedarán los valores de las características después de la transformación. Por cierto, la propiedad de invertibilidad es importante aquí, así que asegúrate de que $P$ sea realmente invertible.

Puedes revisar la lección 'Matrices y operaciones matriciales -> Multiplicación de matrices' para recordar la regla de multiplicación de matrices y su implementación con NumPy.

In [165]:
personal_info_column_list = ['gender', 'age', 'income', 'family_members']
df_pn = df[personal_info_column_list]

In [166]:
df_pn.head()

,gender,age,income,family_members
0,1,41,49600.0,1
1,0,46,38000.0,1
2,0,29,21000.0,0
3,0,21,41700.0,2
4,1,28,26100.0,0


In [167]:
X = df_pn.to_numpy()

La siguiente celda fue creada para reducir el código y para crear una matriz ofuscada invertible de nuestra matriz X para realizar observaciones de ofuscacion y manipulación de metricas

In [168]:
def invertible_matrix(X,seed):
    '''Funcion que crea una matriz aleatoria y la multiplica por una matriz invertible retornando
    nuestra base ofuscada invertible'''
    
    # Creando la matriz aleatoria
    rng = np.random.default_rng(seed=seed)
    P = rng.random(size=(X.shape[1], X.shape[1]))

    # Asegurando que P sea aleatoria
    while np.linalg.det(P) == 0: 
        P = rng.random(size=(X.shape[1], X.shape[1]))

    # Creando la matriz ofuscada (invertible)
    X2_inv = X @ P
    return X2_inv,P

In [169]:
X2_inv, P =invertible_matrix(X,42)

¿Puedes adivinar la edad o los ingresos de los clientes después de la transformación?

No, los datos se han sesgado y a pesar de que el `income` podría entenderse como un valor, es totalmente diferente al valor original. 

In [170]:
encriypted_df = pd.DataFrame(X2_inv, columns=personal_info_column_list)

In [171]:
encriypted_df.head()

,gender,age,income,family_members
0,6359.715273,22380.404676,18424.090742,46000.696690
1,4873.294065,17160.367030,14125.780761,35253.455773
2,2693.117429,9486.397744,7808.831560,19484.860631
3,5345.603937,18803.227203,15479.148373,38663.061863
4,3347.176735,11782.829283,9699.998942,24211.273378


¿Puedes recuperar los datos originales de $X'$ si conoces $P$? Intenta comprobarlo a través de los cálculos moviendo $P$ del lado derecho de la fórmula anterior al izquierdo. En este caso las reglas de la multiplicación matricial son realmente útiles

Siguiendo la logica de la formula si quisieramos obetner X la formula sería la siguiente:
$$ X' = X \cdot P
$$
$$  X' \cdot P^{-1} = X 
$$

In [172]:
def decrypt_matrix (ofuscated_matrix, P):
    '''Esta funcion devuelve la matriz a su dimension original, 
    multiplicando la matriz inversa de P por el la matriz ofuscada'''

    P_inv = np.linalg.inv(P)
    return ofuscated_matrix @ P_inv


In [173]:
decriypted_df = pd.DataFrame(decrypt_matrix(X2_inv, P), columns=personal_info_column_list)

Muestra los tres casos para algunos clientes- Datos originales
- El que está transformado- El que está invertido (recuperado)

In [174]:
print('Dataframe Invertido')
print(decriypted_df.loc[0:2], end='\n\n')
print('Dataframe Recuperado')
print(df_pn.loc[0:2])

Dataframe Invertido
         gender   age   income  family_members
0  1.000000e+00  41.0  49600.0    1.000000e+00
1 -4.473636e-12  46.0  38000.0    1.000000e+00
2 -2.515869e-12  29.0  21000.0    9.524523e-13

Dataframe Recuperado
   gender  age   income  family_members
0       1   41  49600.0               1
1       0   46  38000.0               1
2       0   29  21000.0               0


Seguramente puedes ver que algunos valores no son exactamente iguales a los de los datos originales. ¿Cuál podría ser la razón de ello?

Al multiplicar matrices se hacen calculos que pueden generar valores muy pequeós debido a la precisión finita de los calculos computancionales, la forma de solucionarlo es redondeando los valores para el dataframe y pasarlos a un valor entero

In [175]:
original = decriypted_df.round().astype(int)
inverse = df_pn.round().astype(int)

print(original.loc[0:2], end='\n\n')
print(inverse.loc[0:2])

   gender  age  income  family_members
0       1   41   49600               1
1       0   46   38000               1
2       0   29   21000               0

   gender  age  income  family_members
0       1   41   49600               1
1       0   46   38000               1
2       0   29   21000               0


## Prueba de que la ofuscación de datos puede funcionar con regresión lineal

En este proyecto la tarea de regresión se ha resuelto con la regresión lineal. Tu siguiente tarea es demostrar _analytically_ que el método de ofuscación no afectará a la regresión lineal en términos de valores predichos, es decir, que sus valores seguirán siendo los mismos. ¿Lo puedes creer? Pues no hace falta que lo creas, ¡tienes que que demostrarlo!

Entonces, los datos están ofuscados y ahora tenemos $X \times P$ en lugar de tener solo $X$. En consecuencia, hay otros pesos $w_P$ como
$$
w = (X^T X)^{-1} X^T y \quad \Rightarrow \quad w_P = [(XP)^T XP]^{-1} (XP)^T y
$$

1.- ¿Cómo se relacionarían $w$ y $w_P$ si simplificáramos la fórmula de $w_P$ anterior? 

2.- ¿Cuáles serían los valores predichos con $w_P$? 

3.- ¿Qué significa esto para la calidad de la regresión lineal si esta se mide mediante la RECM?
Revisa el Apéndice B Propiedades de las matrices al final del cuaderno. ¡Allí encontrarás fórmulas muy útiles!

No es necesario escribir código en esta sección, basta con una explicación analítica.

**Respuesta**

1. `Son exactamente la misma formula, sin embargo en lugar de usar X como nuestra matriz a despejar ahora XP (la matriz ofuscada)es la que toma su lugar`

2. Al ser $w_P$ los coeficientes que necesitamos multiplizar para minimizar la diferencia entre las predicciones, `los valores predichos serían la prediccion de una matriz ofuscada`
3. `Esperaríamos que el RMSE sea comparable entre el dataframe ofuscado y el que no.`

**Prueba analítica**

## Prueba de regresión lineal con ofuscación de datos

Ahora, probemos que la regresión lineal pueda funcionar, en términos computacionales, con la transformación de ofuscación elegida.
Construye un procedimiento o una clase que ejecute la regresión lineal opcionalmente con la ofuscación. Puedes usar una implementación de regresión lineal de scikit-learn o tu propia implementación.
Ejecuta la regresión lineal para los datos originales y los ofuscados, compara los valores predichos y los valores de las métricas RMSE y $R^2$. ¿Hay alguna diferencia?

**Procedimiento**

- Crea una matriz cuadrada $P$ de números aleatorios.- Comprueba que sea invertible. Si no lo es, repite el primer paso hasta obtener una matriz invertible.- <¡ tu comentario aquí !>
- Utiliza $XP$ como la nueva matriz de características

Contruir una clase que ejectue la regresion lineal con ofuscacion (opcional)

#### Creando una clase de regresion lineal

In [1]:
class obs_linear_regression():

    def __init__(self, obfuscation=True):
        self.obfuscation = obfuscation
        self.P = None
        '''Constructor linear de regresion
            Parametros:
            - obfuscation (booleano): Indica si la regresion debe utilizar la ofuscación o no.
        '''

    def obfuscate(self, X, seed=40):
        '''Ofusca los datos de entrada utilizando una matriz invertible
            Parametros:
            - X (array): Matriz de caracteristicas de entrada
            - seed (int: Indica La semilla para inicializar números aleatorios
        '''
        if self.obfuscation == True: #revisar porque tiene que ser P
            n = X.shape[0]
            rng = np.random.default_rng(seed=seed)
            self.P = rng.random(size=(X.shape[1], X.shape[1]))
            while np.linalg.det(self.P) == 0: 
                 self.P = rng.random(size=(X.shape[1], X.shape[1]))
            X2 = X @ self.P
            return X2
         
    def fit(self, X, y):
        '''Entrena un modelo de regresion con matrices de caracteristica y de objetivo
            Parametros:
            - X (array): Matriz de caracteristicas de entrada
            - y (array): Matriz Objetivo
        '''
        if self.obfuscation == True:
            X = self.obfuscate(X)
        self.model = LinearRegression().fit(X,y)

    def predict(self,X):
        '''Predice el modelo de regrecion con la matriz de caracteristicas de entrada
            Parametros:
            - X (array): Matriz de caracteristicas de entrada
        '''
        if self.obfuscation == True:
            X = self.obfuscate(X)
        return self.model.predict(X)
    
    def evaluate(self,y,y_predict):
        '''Evalua el modelo y entrega las metricas MSE y RMSE
            Parametros:
            - y (array): Matriz objectivo
            - y_predict (array): Matriz predicha
        '''
        MSE = mean_squared_error(y,y_predict)
        RMSE= np.sqrt(MSE)
        r2 = r2_score(y,y_predict)
        return RMSE, r2

#### Generando una matriz dummy

In [6]:
X = pd.DataFrame([[1,0],[1,4]],columns=['A','B'])
y = pd.DataFrame([1,2], columns=['C'])

#### Ejecutando la clase a través de la función obs_lr

In [28]:
def obs_lr(X,y, obfuscation):
    model = obs_linear_regression(obfuscation=True)
    model.obfuscate(X)
    model.fit(X,y)
    y_pred= model.predict(X)
    RMSE, R2 = model.evaluate(y,y_pred)

    if obfuscation == False:
        print(f'Analisis de  regresion linear para la matriz sin fouscación.', 
              end="\n\n")
    else:
        print(f'Analisis de  regresion linear para la matriz ofuscada. ',
              end="\n\n")
        
    print(f'El valor de RMSE es: {RMSE}')
    print(f'El valor de R2 es: {R2:.2f}', end="\n\n")



#### Resultados

In [29]:
obs_lr(X,y,obfuscation=True)
obs_lr(X,y,False)

Analisis de  regresion linear para la matriz ofuscada. 

El valor de RMSE es: 2.220446049250313e-16
El valor de R2 es: 1.00

Analisis de  regresion linear para la matriz sin fouscación.

El valor de RMSE es: 2.220446049250313e-16
El valor de R2 es: 1.00



`Como podemos observar la ofuscacion de datos no afecta los resultados para las metricas de RMSE y R2`

# Conclusiones

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter.

- [x]  Jupyter Notebook está abierto
- [ ]  El código no tiene errores- [ ]  Las celdas están ordenadas de acuerdo con la lógica y el orden de ejecución
- [ ]  Se ha realizado la tarea 1
    - [ ]  Está presente el procedimiento que puede devolver k clientes similares para un cliente determinado
    - [ ]  Se probó el procedimiento para las cuatro combinaciones propuestas    - [ ]  Se respondieron las preguntas sobre la escala/distancia- [ ]  Se ha realizado la tarea 2
    - [ ]  Se construyó y probó el modelo de clasificación aleatoria para todos los niveles de probabilidad    - [ ]  Se construyó y probó el modelo de clasificación kNN tanto para los datos originales como para los escalados. Se calculó la métrica F1.- [ ]  Se ha realizado la tarea 3
    - [ ]  Se implementó la solución de regresión lineal mediante operaciones matriciales    - [ ]  Se calculó la RECM para la solución implementada- [ ]  Se ha realizado la tarea 4
    - [ ]  Se ofuscaron los datos mediante una matriz aleatoria e invertible P    - [ ]  Se recuperaron los datos ofuscados y se han mostrado algunos ejemplos    - [ ]  Se proporcionó la prueba analítica de que la transformación no afecta a la RECM    - [ ]  Se proporcionó la prueba computacional de que la transformación no afecta a la RECM- [ ]  Se han sacado conclusiones

# Apéndices

## Apéndice A: Escribir fórmulas en los cuadernos de Jupyter

Puedes escribir fórmulas en tu Jupyter Notebook utilizando un lenguaje de marcado proporcionado por un sistema de publicación de alta calidad llamado $\LaTeX$ (se pronuncia como "Lah-tech"). Las fórmulas se verán como las de los libros de texto.

Para incorporar una fórmula a un texto, pon el signo de dólar (\\$) antes y después del texto de la fórmula, por ejemplo: $\frac{1}{2} \times \frac{3}{2} = \frac{3}{4}$ or $y = x^2, x \ge 1$.

Si una fórmula debe estar en el mismo párrafo, pon el doble signo de dólar (\\$\\$) antes y después del texto de la fórmula, por ejemplo:
$$
\bar{x} = \frac{1}{n}\sum_{i=1}^{n} x_i.
$$

El lenguaje de marcado de [LaTeX](https://es.wikipedia.org/wiki/LaTeX) es muy popular entre las personas que utilizan fórmulas en sus artículos, libros y textos. Puede resultar complicado, pero sus fundamentos son sencillos. Consulta esta [ficha de ayuda](http://tug.ctan.org/info/undergradmath/undergradmath.pdf) (materiales en inglés) de dos páginas para aprender a componer las fórmulas más comunes.

## Apéndice B: Propiedades de las matrices

Las matrices tienen muchas propiedades en cuanto al álgebra lineal. Aquí se enumeran algunas de ellas que pueden ayudarte a la hora de realizar la prueba analítica de este proyecto.

| Distributividad                                   | $A(B+C)=AB+AC$            |
|---------------------------------------------------|----------------------------|
| No conmutatividad                                 | $AB \neq BA$               |
| Propiedad asociativa de la multiplicación         | $(AB)C = A(BC)$            |
| Propiedad de identidad multiplicativa             | $IA = AI = A$              |
|                                                   | $A^{-1}A = AA^{-1} = I$    |
|                                                   | $(AB)^{-1} = B^{-1}A^{-1}$ |
| Reversibilidad de la transposición de un producto de matrices | $(AB)^T = B^TA^T$ |
